In [ ]:
pip install pandas nltk textblob scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from textblob import TextBlob

In [ ]:
df_makanan = pd.read_csv("https://raw.githubusercontent.com/SyahrezaAdnanAlAzhar/tugasBesarLabAI/main/makanan_sehat_training.csv")
df_makanan_validation = pd.read_csv("https://raw.githubusercontent.com/SyahrezaAdnanAlAzhar/tugasBesarLabAI/main/makanan_sehat_validation.csv")

In [ ]:
df_makanan

,text,sentiment,clean_text
0,Saya baru saja diagnosa dengan kadar gula dara...,makanan rendah kalori,saya baru saja diagnosa dengan kadar gula dara...
1,Sedang menjalani program penurunan berat badan...,makanan rendah kalori,sedang menjalani program penurunan berat badan...
2,Baru saja diberi tahu bahwa kadar kolesterol s...,makanan rendah kalori,baru saja diberi tahu bahwa kadar kolesterol s...
3,Saya sedang berjuang dengan obesitas dan ingin...,makanan rendah kalori,saya sedang berjuang dengan obesitas dan ingin...
4,Mengalami masalah pencernaan dan dokter menyar...,makanan rendah kalori,mengalami masalah pencernaan dan dokter menyar...
...,...,...,...
695,Saya baru saja pulang dari perjalanan di pegun...,makanan untuk menjaga kesehatan mata,saya baru saja pulang dari perjalanan di pegun...
696,Mata saya seringkali terasa perih setelah bere...,makanan untuk menjaga kesehatan mata,mata saya seringkali terasa perih setelah bere...
697,Saya merasa penglihatan saya seringkali kabur ...,makanan untuk menjaga kesehatan mata,saya merasa penglihatan saya seringkali kabur ...
698,Mata saya seringkali terasa panas dan berat se...,makanan untuk menjaga kesehatan mata,mata saya seringkali terasa panas dan berat se...


In [ ]:
df_makanan_validation

,text,sentiment,clean_text
0,Kondisi alergi makanan membuat saya mencari ma...,makanan rendah kalori,kondisi alergi makanan membuat saya mencari ma...
1,Penderita maag mencari makanan rendah kalori y...,makanan rendah kalori,penderita maag mencari makanan rendah kalori y...
2,Baru saja pulih dari infeksi saluran pernapasa...,makanan rendah kalori,baru saja pulih dari infeksi saluran pernapasa...
3,Kondisi perut kembung membuat saya mencari mak...,makanan rendah kalori,kondisi perut kembung membuat saya mencari mak...
4,Saya baru saja menjalani operasi jantung dan b...,makanan rendah kalori,saya baru saja menjalani operasi jantung dan b...
...,...,...,...
415,Mata saya sering kali mengalami ketegangan set...,makanan untuk menjaga kesehatan mata,mata saya sering kali mengalami ketegangan set...
416,Saya mengalami gangguan penglihatan malam hari...,makanan untuk menjaga kesehatan mata,saya mengalami gangguan penglihatan malam hari...
417,Kondisi mata saya cenderung kering dan terasa ...,makanan untuk menjaga kesehatan mata,kondisi mata saya cenderung kering dan terasa ...
418,Saya baru saja pulih dari infeksi mata. dan do...,makanan untuk menjaga kesehatan mata,saya baru saja pulih dari infeksi mata dan dok...


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

# Preprocessing Teks
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    words = word_tokenize(text)
    filtered_words = [word.lower() for word in words if word.isalpha() and word.lower() not in stop_words]
    return ' '.join(filtered_words)

df_makanan['clean_text'] = df_makanan['text'].apply(preprocess_text)
df_makanan_validation['clean_text'] = df_makanan_validation['text'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Menggunakan dataset yang telah dibagi dari file validasi
X_train, y_train = df_makanan['clean_text'], df_makanan['sentiment']
X_test, y_test = df_makanan_validation['clean_text'], df_makanan_validation['sentiment']

In [ ]:
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:
# Model Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train)

MultinomialNB()

In [ ]:
# Evaluasi Model
y_pred = nb_model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{report}')

In [ ]:
# Fungsi untuk prediksi sentimen dengan empat kelas
def predict_sentiment(text):
    cleaned_text = preprocess_text(text)
    vectorized_text = tfidf_vectorizer.transform([cleaned_text])
    prediction = nb_model.predict(vectorized_text)[0]

    return prediction

In [ ]:
# Uji Model dengan Input Pengguna
user_input = input("Masukkan teks untuk diuji sentimennya: ")
prediction_output = predict_sentiment(user_input)
print(f'Prediksi Sentimen: {prediction_output}')

Masukkan teks untuk diuji sentimennya: makanan yang bikin kulit aku makin bersih dan ganteng
Prediksi Sentimen: makanan kaya karbohidrat


In [ ]:
df_response = pd.read_excel("https://docs.google.com/spreadsheets/d/1WIHxAdAIhbcamQH0bBQ8eF6Pv2OD32tvS9bnAFqtgC4/export?format=xlsx")

In [ ]:
df_response

,sentiment,response
0,makanan rendah kalori,Sebuah opsi camilan yang sehat dan rendah kalo...
1,makanan rendah kalori,Cobalah camilan yoghurt beku rendah lemak. Ras...
2,makanan rendah kalori,Sebuah opsi makan malam sehat dan rendah kalor...
3,makanan rendah kalori,Cobalah sarapan oatmeal dengan potongan buah s...
4,makanan rendah kalori,Bagaimana dengan pilihan protein tanpa lemak s...
...,...,...
65,makanan untuk menjaga kesehatan mata,"Untuk menjaga kesehatan mata, Anda sebaiknya m..."
66,makanan untuk menjaga kesehatan mata,"Selain wortel, bayam, dan ikan salmon, Anda ju..."
67,makanan untuk menjaga kesehatan mata,"Untuk kepraktisan, Anda bisa mengonsumsi telur..."
68,makanan untuk menjaga kesehatan mata,"Untuk makanan ringan yang baik untuk mata, And..."


In [ ]:
def get_random_response(sentiment):
    relevant_responses = df_response[df_response['sentiment'] == sentiment]['response']
    if not relevant_responses.empty:
        return np.random.choice(relevant_responses)
    else:
        return "Maaf, kami belum dapat merekomendasikan makanan kepada anda."

In [ ]:
user_input = input("Masukkan teks untuk diuji sentimennya: ")
prediction_output = predict_sentiment(user_input)
random_response = get_random_response(prediction_output)
print(f"Prediksi Sentimen: {prediction_output}")
print(f"Respons: {random_response}")

Masukkan teks untuk diuji sentimennya: aku lagi asam lambung
Prediksi Sentimen: makanan kaya karbohidrat
Respons: Anda bisa mencoba smoothie bowl dengan campuran buah-buahan seperti mangga, kiwi, dan alpukat. Sajikan dengan granola untuk tambahan karbohidrat yang sehat.
